In [15]:
import numpy as np
import pandas as pd
from scipy.sparse import *
from tqdm import tqdm
from collections import Counter
from itertools import chain, combinations
import time
from gensim.models import Word2Vec
tqdm.pandas()

In [2]:
#플레이 리스트
train = pd.read_json('data/train.json')

#곡 정보
song_meta = pd.read_json('data/song_meta.json')

In [3]:
display(train.head(1))
display(song_meta.head(1))

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


In [4]:
# 노래 id가 key, 값은 그 노래를 부른 가수 리스트
songid_to_artists = dict(zip(song_meta['id'],song_meta['artist_name_basket']))

In [5]:
# 각 플레이리스트에 포함된 노래를 부른 가수들을 가져오기 위한 함수
def get_artist(playlist):
    artists = set()
    for song in playlist:
        artists.update(songid_to_artists[song])
    return list(artists)

In [8]:
train['artists'] = train['songs'].progress_apply(get_artist)

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:04<00:00, 24555.20it/s]


In [85]:
list(train[['artists']].iloc[0].iteritems())

[('artists',
  ['Phoenix',
   'Never Shout Never',
   'Honeymoon Suite',
   'Royal Bangs',
   'Bruno Mars',
   'Cody Simpson',
   'OneRepublic',
   'The Sol',
   'Lily Of The Valley',
   'Red Hot Chili Peppers',
   'Corinne Bailey Rae',
   'Tomi Swick',
   'Boyzone',
   'All About She',
   'Man Man',
   'Christophe Mae',
   'James Blunt'])]

In [12]:
#플레이리스트 내 가수 수의 최대값
train['artists'].apply(lambda x : len(x)).max()

250

In [53]:
#10분 정도 걸림
# w2v_model = Word2Vec(train['artists'].tolist(), min_count = 1, size = 100 , window = 200, sg = 1)
# w2v_model.save('artist_w2v.model')

In [80]:
w2v_model.wv.most_similar('마마무 (Mamamoo)')

[('WINNER', 0.9203924536705017),
 ('San E', 0.885474443435669),
 ('스피카', 0.8853033781051636),
 ('KissBlue (키스블루)', 0.8834551572799683),
 ('걸크러쉬 (GIRL CRUSH)', 0.8833224177360535),
 ('레이나 (Raina)', 0.8829860687255859),
 ('배희관 밴드', 0.882705807685852),
 ('정은지', 0.8822885751724243),
 ('Leah Dizon', 0.8819307088851929),
 ('수지 (SUZY)', 0.8817796111106873)]

In [74]:
#가수 벡터들을 df로 변환
artist_w2v_dict = {w: w2v_model.wv[w] for w in w2v_model.wv.vocab}
artist_w2v_df = pd.DataFrame(artist_w2v_dict).T

In [77]:
# 저장
artist_w2v_df.to_csv('data/artist_w2v_df.csv')